In [1]:
import pandas as pd
from metric import pr_score

In [2]:
test = "tra0_gpt-4o_10-2-17"
response = pd.read_csv(f'chatgpt/responses/{test}/{test}_summary_formatted.csv')
equations = []
answers = []
for i, row in response.iterrows():
    for j in range(2, 17):
        equations.append(row[f'equation_{{{j}}}'][:-1])
        if pd.isna(row[f'answer_{{{j}}}']):
            answers.append([])
        else:
            answers.append(row[f'answer_{{{j}}}'].split("="))

In [3]:
from utils import _check_equation_format, _check_number_format, check_format

# check if response initial equation doesn't match actual equation
print("responses where equation doesn't match original equation: ")
for i, equation in enumerate(equations):
    if len(answers[i]) == 0 or equation != answers[i][0]:
        print(equation, answers[i])

# check format of each equation and response
print("equations or responses that have incorrect formats: ")
for equation in equations:
    try:
        _check_equation_format(equation)
    except:
        print(equation)
for answer in answers:
    for equation in answer:
        if len(check_format(equation)) == 0:
            print(equation)

responses where equation doesn't match original equation: 
1001010101011000_{2}+11011111101011_{2} []
equations or responses that have incorrect formats: 


In [4]:
for answer in answers:
    print(answer)

['110001010011111_{2}+110101110101010_{2}', '10011110111101101_{2}']
['1021122002_{3}+1101210211_{3}', '21210202010_{3}']
['12022133_{4}+12232222_{4}', '25247_{10}+27442_{10}', '52689_{10}', '30131301_{4}']
['1301442_{5}+1340222_{5}', '25247_{10}+27662_{10}', '52909_{10}', '3104134_{5}']
['312515_{6}+331334_{6}', '25247_{10}+27562_{10}', '52809_{10}', '1044253_{6}']
['133415_{7}+143233_{7}', '25247_{10}+27562_{10}', '52809_{10}', '306651_{7}']
['61237_{8}+65652_{8}', '25247_{10}+27602_{10}', '52849_{10}', '147161_{8}']
['37562_{9}+41724_{9}', '25247_{10}+27562_{10}', '52809_{10}', '80386_{9}']
['25247_{10}+27562_{10}', '52809_{10}']
['17A72_{11}+19787_{11}', '25247_{10}+27562_{10}', '52809_{10}', '36947_{11}']
['1273B_{12}+13B4A_{12}', '25247_{10}+27562_{10}', '52809_{10}']
['B651_{13}+C712_{13}', '25247_{10}+27562_{10}', '52809_{10}', '1B049_{13}']
['92B5_{14}+A08A_{14}', '25247_{10}+27562_{10}', '52809_{10}', '15361_{14}']
['7732_{15}+8277_{15}', '25247_{10}+27562_{10}', '52809_{10}'

In [5]:
score, _ = pr_score(equations, answers, pr_ratio=0)
print("pr_ratio=0:\t\t", score)
score, graded = pr_score(equations, answers)
print("pr_ratio=0.4:\t", score)
score, _ = pr_score(equations, answers, pr_ratio=1)
print("pr_ratio=1:\t\t", score)
print("y_graded keys:", graded[equations[0]].keys())

pr_ratio=0:		 0.32
pr_ratio=0.4:	 0.4399999999999996
pr_ratio=1:		 0.6200000000000003
y_graded keys: dict_keys(['y_process', 'x_correctness', 'y_correctness', 'process_score', 'pr_score(0.4)'])


In [6]:
for x, y_graded in zip(equations, (graded[equation] for equation in equations)):
    copy = y_graded.copy()
    copy['x'] = x
    copy.pop('y_process')
    print(copy)
    display(y_graded['y_process'])

{'x_correctness': 1, 'y_correctness': 0, 'process_score': 0.0, 'pr_score(0.4)': 0.0, 'x': '110001010011111_{2}+110101110101010_{2}'}


,equation,result,correct_result =,correctness
0,110001010011111_{2}+110101110101010_{2},10011110111101101_{2},1100111001001001_{2},0.0


{'x_correctness': 1, 'y_correctness': 0, 'process_score': 0.0, 'pr_score(0.4)': 0.0, 'x': '1021122002_{3}+1101210211_{3}'}


,equation,result,correct_result =,correctness
0,1021122002_{3}+1101210211_{3},21210202010_{3},2200102220_{3},0.0


{'x_correctness': 1, 'y_correctness': 0, 'process_score': 0.3333333333333333, 'pr_score(0.4)': 0.13333333333333333, 'x': '12022133_{4}+12232222_{4}'}


...

,equation,result,correct_result =,correctness
0,5DA9_{14}+CA10_{14},14919_{14},149B9_{14},0.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '4CE7_{15}+A51C_{15}'}


,equation,result,correct_result =,correctness
0,4CE7_{15}+A51C_{15},16417_{10}+34902_{10},16417_{10}+34902_{10},1.0
1,16417_{10}+34902_{10},51319_{10},51319_{10},1.0
2,51319_{10},10314_{15},10314_{15},1.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '4021_{16}+8856_{16}'}


,equation,result,correct_result =,correctness
0,4021_{16}+8856_{16},16417_{10}+34902_{10},16417_{10}+34902_{10},1.0
1,16417_{10}+34902_{10},51319_{10},51319_{10},1.0
2,51319_{10},C877_{16},C877_{16},1.0


In [7]:
for x, y_graded in zip(equations, (graded[equation] for equation in equations)):
    if y_graded['y_correctness'] == 1:
        copy = y_graded.copy()
        copy['x'] = x
        copy.pop('y_process')
        print(copy)
        display(y_graded['y_process'])

{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '312515_{6}+331334_{6}'}


,equation,result,correct_result =,correctness
0,312515_{6}+331334_{6},25247_{10}+27562_{10},25247_{10}+27562_{10},1.0
1,25247_{10}+27562_{10},52809_{10},52809_{10},1.0
2,52809_{10},1044253_{6},1044253_{6},1.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '133415_{7}+143233_{7}'}


,equation,result,correct_result =,correctness
0,133415_{7}+143233_{7},25247_{10}+27562_{10},25247_{10}+27562_{10},1.0
1,25247_{10}+27562_{10},52809_{10},52809_{10},1.0
2,52809_{10},306651_{7},306651_{7},1.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '37562_{9}+41724_{9}'}


...

,equation,result,correct_result =,correctness
0,9601_{12}+18246_{12},51319_{10},25847_{12},0.0
1,51319_{10},25847_{12},25847_{12},1.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '4CE7_{15}+A51C_{15}'}


,equation,result,correct_result =,correctness
0,4CE7_{15}+A51C_{15},16417_{10}+34902_{10},16417_{10}+34902_{10},1.0
1,16417_{10}+34902_{10},51319_{10},51319_{10},1.0
2,51319_{10},10314_{15},10314_{15},1.0


{'x_correctness': 1, 'y_correctness': 1, 'process_score': 1.0, 'pr_score(0.4)': 1.0, 'x': '4021_{16}+8856_{16}'}


,equation,result,correct_result =,correctness
0,4021_{16}+8856_{16},16417_{10}+34902_{10},16417_{10}+34902_{10},1.0
1,16417_{10}+34902_{10},51319_{10},51319_{10},1.0
2,51319_{10},C877_{16},C877_{16},1.0
